We will use the "Concrete Crack Images for Classification" Dataset first below is the link to the dataset
https://data.mendeley.com/datasets/5y9wdsg2zt/2

Let us first split the dataset into train val and test

In [ ]:
import random
import shutil
import os
import glob
##the data path should be the path to the folder where the imeediate subfolders are the classes
img_path = "data/Concrete_Crack_Images_for_Classification"
data_path = "data"
train_path = data_path+"/train"
val_path = data_path+"/val"
test_path = data_path+"/test"

#if you are running it again we dont need to split the data so this is just the check 
data_path_paths = glob.glob(img_path+'/**/*.jpg',recursive=True)
if len(data_path_paths)>0:
    #we will make a 80% 10% 10% split
    #######################################################
    for clss in os.listdir(img_path):
        img_paths = glob.glob(img_path+'/'+clss+"/**/*.jpg",recursive=True)
        random.shuffle(img_paths)
        os.makedirs(train_path+'/'+clss,exist_ok=True)
        for img_p in img_paths[:int(len(img_paths)*0.80)]:
            shutil.move(img_p,train_path+'/'+clss)
        os.makedirs(val_path+'/'+clss,exist_ok=True)
        for img_p in img_paths[int(len(img_paths)*0.80):int(len(img_paths)*0.90)]:
            shutil.move(img_p,val_path+'/'+clss)
        os.makedirs(test_path+'/'+clss,exist_ok=True)
        for img_p in img_paths[int(len(img_paths)*0.90):]:
            shutil.move(img_p,test_path+'/'+clss)
    

here we define the train cycle and val cycle

In [ ]:
import torch
def train_cycle(train_dataloader,optimizer,criterion,model):
    model.train()
    epoch_loss = 0
    epochs_acc = 0
    count = 0
    for images,labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        epoch_loss+=loss.item()
        _,predicted=torch.max(outputs.data,1)
        count+=labels.size(0)
        epochs_acc+=(predicted==labels).sum().item()
    epoch_loss=epoch_loss/len(train_dataloader)
    epochs_acc = epochs_acc/count
    return epoch_loss,epochs_acc

def val_cycle(val_dataloader,criterion,model):
    model.eval()
    epoch_loss = 0
    epochs_acc = 0
    count = 0
    with torch.no_grad():
        for images,labels in val_dataloader:
            outputs = model(images)
            loss = criterion(outputs,labels)
            epoch_loss+=loss.item()
            _,predicted=torch.max(outputs.data,1)
            count+=labels.size(0)
            epochs_acc+=(predicted==labels).sum().item()
        epoch_loss=epoch_loss/len(val_dataloader)
        epochs_acc = epochs_acc/count
    return epoch_loss,epochs_acc

Here we will define the training settings

In [ ]:
num_of_epochs = 30
batch_size = 8
learning_rate=0.001

we first load the dataset

In [ ]:
import torchvision
from torch.utils.data import DataLoader
#for the transforms we will not be resizing the images since this dataset already has resized images

transforms = torchvision.transforms.ToTensor()
train_dataset = torchvision.datasets.ImageFolder(train_path,transform=transforms)
val_dataset = torchvision.datasets.ImageFolder(val_path,transform=transforms)
test_dataset = torchvision.datasets.ImageFolder(test_path,transform=transforms)

train_dataloader = DataLoader(train_dataset,batch_size=batch_size,num_workers=4,shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size=batch_size,num_workers=4,shuffle=False)
train_dataloader = DataLoader(train_dataset,batch_size=batch_size,num_workers=4,shuffle=False)

First we will benchmark the effecientnet_b0 model without the stochastic depth layer

In [ ]:
from Architectures.eficientnetb0 import efficientnet_b0_without_stochastic
import torch.optim as optim
import torch.nn as nn

model_without = efficientnet_b0_without_stochastic(num_classes=len(train_dataset.classes))

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model_without.parameters(), lr=learning_rate)


In [ ]:
train_loss_list = []
train_acc_list = []
valid_loss_list = []
valid_acc_list = []

for epoch in range(num_of_epochs):
    train_epoch_loss,train_epoch_acc=train_cycle(train_dataloader,optimizer,criterion,model_without)
    val_epoch_loss,val_epoch_acc=val_cycle(val_dataloader,optimizer,criterion,model_without)
    train_loss_list.append(train_epoch_loss)
    train_acc_list.append(train_epoch_acc)
    valid_loss_list.append(val_epoch_loss)
    valid_acc_list.append(val_epoch_acc)
    